# Plotly Dash Dashboard

This dashboard allows interactive exploration of the AI Text Detector results. It has three tabs for:
1. **EDA** – Visualizing data distributions (class balance, article length, etc.).
2. **Evaluation** – Showing model performance metrics (confusion matrix, ROC curves).
3. **Inference** – Providing an interface to input new text and get predictions with explanations.

Run this app to launch the dashboard locally and interact with the model.


In [5]:
import dash
from dash import dcc, html
import base64, logging
from pathlib import Path
from typing import Optional

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "AI Text Detector Dashboard"

# ——— 1) Auto‑locate your diagrams/ directory ————————————————
def find_diagrams_dir(start: Path = Path.cwd(), marker: str = "diagrams") -> Optional[Path]:
    """
    Walk upward from `start` through its parents until you find a folder named `marker`.
    Return the Path to that folder, or None if not found.
    """
    for folder in (start, *start.parents):
        candidate = folder / marker
        if candidate.is_dir():
            return candidate
    logging.warning(f"Couldn’t locate a '{marker}/' directory under {start}")
    return None

DIAGRAMS_DIR = find_diagrams_dir()

# ——— 2) Helper to read & encode image files as base64 URIs, safely ——————
def encode_image(filename: str) -> Optional[str]:
    """
    Given a filename (e.g. "class_distribution.png"), look in DIAGRAMS_DIR,
    read it if present, and return a data URI. If missing, log & return None.
    """
    if DIAGRAMS_DIR is None:
        return None

    img_path = (DIAGRAMS_DIR / filename).resolve()
    if not img_path.exists():
        logging.warning(f"Image not found, skipping: {img_path}")
        return None

    try:
        raw = img_path.read_bytes()
        b64 = base64.b64encode(raw).decode("utf-8")
        return f"data:image/png;base64,{b64}"
    except Exception as e:
        logging.warning(f"Failed to encode {img_path}: {e}")
        return None

# ——— 3) Load pre‑generated plots (won’t crash if missing) ————————
class_dist_img  = encode_image("class_distribution.png")
length_dist_img = encode_image("length_distribution.png")
conf_matrix_img = encode_image("confusion_matrix.png")
roc_curves_img  = encode_image("roc_curves.png")

# ——— 4) Define app layout ————————————————————————————————
app.layout = html.Div([
    html.H1(
        "AI‑Generated Text Detection Dashboard",
        style={"textAlign": "center", "marginBottom": "1em"}
    ),
    dcc.Tabs(id="tabs", value="tab-eda", children=[
        dcc.Tab(label="EDA",        value="tab-eda"),
        dcc.Tab(label="Evaluation", value="tab-eval"),
        dcc.Tab(label="Inference",  value="tab-inf"),
    ]),
    html.Div(id="tab-content")
])

# (Callbacks will go here in later cells…)


In [6]:
from dash.dependencies import Input, Output

@app.callback(Output("tab-content", "children"), Input("tabs", "value"))
def render_tab_content(tab):
    if tab == "tab-eda":
        return html.Div([
            html.H3("Exploratory Data Analysis", style={"textAlign": "center", "marginTop": "1em"}),
            html.Img(src=class_dist_img, style={"width": "45%", "display": "inline-block", "padding": "1em"}),
            html.Img(src=length_dist_img, style={"width": "45%", "display": "inline-block", "padding": "1em"}),
            html.P(
                "The dataset is fairly balanced across classes. "
                "Human-written texts are generally longer than AI-generated or AI-paraphrased ones.",
                style={"textAlign": "center", "fontStyle": "italic", "marginTop": "0.5em"}
            )
        ])
    elif tab == "tab-eval":
        # Evaluation tab: show confusion matrix and ROC curves
        return html.Div([
            html.H3("Model Evaluation", style={"textAlign": "center", "marginTop": "1em"}),
            html.Img(src=conf_matrix_img, style={"width": "40%", "display": "inline-block", "padding": "1em"}),
            html.Img(src=roc_curves_img, style={"width": "50%", "display": "inline-block", "padding": "1em"}),
            html.P(
                "Overall accuracy ~91%. The model excels at identifying human-written text (near 99% recall) "
                "and mostly confuses paraphrased vs directly AI-generated text.",
                style={"textAlign": "center", "fontStyle": "italic", "marginTop": "0.5em"}
            )
        ])
    elif tab == "tab-inf":
        # Inference tab: textarea + button + placeholder for results
        return html.Div([
            html.H3("Try the Detector", style={"textAlign": "center", "marginTop": "1em"}),
            dcc.Textarea(
                id="input-text",
                placeholder="Enter article text here...",
                style={"width": "80%", "height": "100px"}
            ),
            html.Br(),
            html.Button("Detect", id="detect-button", n_clicks=0, style={"marginTop": "0.5em"}),
            html.Div(id="result-output", style={"marginTop": "1em"})
        ])
    return html.Div()


In [7]:
if __name__ == "__main__":
    app.run_server(debug=True)
